In [13]:
import cv2
import numpy as np
import os
import pandas as pd

from sklearn.model_selection import train_test_split
from skimage import exposure

train_images     = 'train_ims'
train_label_file = 'train.csv'

def load_train_data(dataset_path, csv_path):
    '''
    Load the image data and labels.
    '''
    
    df = pd.read_csv(csv_path)
    images = []
    labels = []
    
    for index, row in df.iterrows():
        image_name = row['im_name']
        label      = row['label']
        
        image_path  = os.path.join(dataset_path, image_name)
        image       = cv2.imread(image_path)
        images.append(image)
        labels.append(label)
        
    return images, labels

images, labels = load_train_data(train_images, train_label_file)

In [14]:
print(images[0], labels[0])

[[[246 242 237]
  [249 246 238]
  [241 239 228]
  ...
  [ 25  62  28]
  [ 35  66  33]
  [ 37  67  32]]

 [[251 245 238]
  [242 238 227]
  [237 234 220]
  ...
  [ 29  66  32]
  [ 27  60  26]
  [ 28  60  25]]

 [[228 222 209]
  [220 216 198]
  [214 210 191]
  ...
  [ 29  71  36]
  [ 35  72  38]
  [ 28  63  29]]

 ...

 [[219 205 177]
  [210 196 167]
  [199 189 155]
  ...
  [161 156 117]
  [166 172 131]
  [117 127  84]]

 [[210 197 171]
  [203 191 163]
  [186 177 143]
  ...
  [148 145 107]
  [128 136  95]
  [118 132  90]]

 [[229 218 191]
  [219 208 180]
  [193 184 150]
  ...
  [153 150 112]
  [135 144 104]
  [122 139  96]]] 6
